# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
geo = [
    pd.read_csv('C:\\Users\\admin\\Desktop\\DS_yandex\\07_Выбор локации для скважины\\geo_data_0.csv'), 
    pd.read_csv('C:\\Users\\admin\\Desktop\\DS_yandex\\07_Выбор локации для скважины\\geo_data_1.csv'), 
    pd.read_csv('C:\\Users\\admin\\Desktop\\DS_yandex\\07_Выбор локации для скважины\\geo_data_2.csv')
]

In [3]:
display(geo[0].head())
geo[0].info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
dublic = list(geo[0]['id'].value_counts().head(10).index)
print(geo[0].query('id in @dublic').sort_values(by = 'id'))

          id        f0        f1         f2     product
66136  74z30  1.084962 -0.312358   6.990771  127.643327
64022  74z30  0.741456  0.459229   5.153109  140.771492
51970  A5aEY -0.180335  0.935548  -2.094773   33.020205
3389   A5aEY -0.039949  0.156872   0.209861   89.249364
69163  AGS9W -0.933795  0.116194  -3.655896   19.230453
42529  AGS9W  1.454747 -0.479651   0.683380  126.370504
931    HZww2  0.755284  0.368511   1.863211   30.681774
7530   HZww2  1.061194 -0.373969  10.430210  158.828695
63593  QcMuo  0.635635 -0.473422   0.862670   64.578675
1949   QcMuo  0.506563 -0.323775  -2.215583   75.496502
75715  Tdehs  0.112079  0.430296   3.218993   60.964018
21426  Tdehs  0.829407  0.298807  -0.049563   96.035308
92341  TtcGQ  0.110711  1.022689   0.911381  101.318008
60140  TtcGQ  0.569276 -0.104876   6.440215   85.350186
89582  bsk9y  0.398908 -0.400253  10.122376  163.433078
97785  bsk9y  0.378429  0.005837   0.160827  160.637302
41724  bxg6G -0.823752  0.546319   3.630479   93

In [5]:
display(geo[1].head())
geo[1].info()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
dublic = list(geo[1]['id'].value_counts().head(4).index)
print(geo[1].query('id in @dublic').sort_values(by = 'id'))

          id         f0         f1        f2     product
5849   5ltQ6  -3.435401 -12.296043  1.999796   57.085625
84461  5ltQ6  18.213839   2.191999  3.993869  107.813044
1305   LHZR0  11.170835  -1.945066  3.002872   80.859783
41906  LHZR0  -8.989672  -4.286607  2.009139   57.085625
2721   bfPNe  -9.494442  -5.463692  4.006042  110.992147
82178  bfPNe  -6.202799  -4.820045  2.995107   84.038886
47591  wt4Uk  -9.091098  -8.109279 -0.002314    3.179103
82873  wt4Uk  10.259972  -9.376355  4.994297  134.766305


In [7]:
display(geo[2].head())
geo[2].info()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
dublic = list(geo[2]['id'].value_counts().head(4).index)
print(geo[2].query('id in @dublic').sort_values(by = 'id'))

          id        f0        f1        f2     product
45404  KUPhW  0.231846 -1.698941  4.990775   11.716299
55967  KUPhW  1.211150  3.176408  5.543540  132.831802
11449  VF7Jo  2.122656 -0.858275  5.746001  181.716817
49564  VF7Jo -0.883115  0.560537  0.723601  136.233420
44378  Vcm5J -1.229484 -2.439204  1.222909  137.968290
95090  Vcm5J  2.587702  1.986875  2.482245   92.327572
28039  xCHr8  1.633027  0.368135 -2.378367    6.120525
43233  xCHr8 -0.847066  2.101796  5.597130  184.388641


### Замечания по предоставленным данным

Во всех трёх наборах данных присутствует повторения id скважин, однако остальные данные по этим скважинам разнятся. Вероятнее всего при выгрузке данных произошол сбой и реальные id данных скважин не попали в наборы данных. Необходимо сообщить об этом тем, что ответственнен за выгрузку.

## Обучение и проверка модели

In [9]:
def train_model(n):
    features = geo[n].drop(['id', 'product'], axis = 1)
    target = geo[n]['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                            target, 
                                                                            test_size = 0.25, 
                                                                            random_state = 12345)
    
    model = LinearRegression()
    model.fit(features_train,target_train) 
    predicted_valid = model.predict(features_valid)
    mean = predicted_valid.mean()
    rmse = (mean_squared_error(target_valid, predicted_valid) ** 0.5)
    r2 = r2_score(target_valid, predicted_valid)
    data = {'model': model, 
            'target_valid': target_valid.reset_index(drop = True), 
            'predicted_valid': predicted_valid, 
            'r2': r2, 
            'rmse': rmse, 
            'mean': mean}
    return data

### Регион "0"

In [10]:
model_geo_0 = train_model(0)

In [11]:
print('Средний запас предсказанного сырья:', model_geo_0['mean'].round(3), 
      '\nЗначение RMSE:', model_geo_0['rmse'].round(3), 
      '\nЗначение R2-score:', model_geo_0['r2'].round(3))

Средний запас предсказанного сырья: 92.593 
Значение RMSE: 37.579 
Значение R2-score: 0.28


### Регион "1"

In [12]:
model_geo_1 = train_model(1)

In [13]:
print('Средний запас предсказанного сырья:', model_geo_1['mean'].round(3), 
      '\nЗначение RMSE:', model_geo_1['rmse'].round(3), 
      '\nЗначение R2-score:', model_geo_1['r2'].round(3))

Средний запас предсказанного сырья: 68.729 
Значение RMSE: 0.893 
Значение R2-score: 1.0


### Регион "2"

In [14]:
model_geo_2 = train_model(2)

In [15]:
print('Средний запас предсказанного сырья:', model_geo_2['mean'].round(3), 
      '\nЗначение RMSE:', model_geo_2['rmse'].round(3), 
      '\nЗначение R2-score:', model_geo_2['r2'].round(3))

Средний запас предсказанного сырья: 94.965 
Значение RMSE: 40.03 
Значение R2-score: 0.205


### Выводы

Для каждого региона модели были обучены и с необходимыми далее переменными сохранены.
Следует отметить, что у региона "1" очень низкое значение RMSE и R2-score равен единице.

## Подготовка к расчёту прибыли

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [16]:
BUDGET = 10*(10**9)
TOP_200 = 200
PRICE_BARREL = 450000

In [17]:
product_no_loss = round(BUDGET/TOP_200/PRICE_BARREL, 2)
print(product_no_loss)

111.11


Сравним полученный объем сырья со средними запасами по регионам.

In [18]:
for i in range(3):
    display(geo[i]['product'].describe())

count    100000.000000
mean         92.500000
std          44.288691
min           0.000000
25%          56.497507
50%          91.849972
75%         128.564089
max         185.364347
Name: product, dtype: float64

count    100000.000000
mean         68.825000
std          45.944423
min           0.000000
25%          26.953261
50%          57.085625
75%         107.813044
max         137.945408
Name: product, dtype: float64

count    100000.000000
mean         95.000000
std          44.749921
min           0.000000
25%          59.450441
50%          94.925613
75%         130.595027
max         190.029838
Name: product, dtype: float64

In [19]:
for i in range(3):
    print('Средний объём нефти в скажинах', i, 'региона равен', geo[i]['product'].mean().round(6), 'баррелей.')

Средний объём нефти в скажинах 0 региона равен 92.5 баррелей.
Средний объём нефти в скажинах 1 региона равен 68.825 баррелей.
Средний объём нефти в скажинах 2 региона равен 95.0 баррелей.


In [20]:
print('36% скважин в регионе geo_0 содержат больше сырья, чем', round(geo[0]['product'].quantile(0.64),2), 'баррелей.')
print('17% скважин в регионе geo_1 содержат больше сырья, чем', round(geo[1]['product'].quantile(0.83),2), 'баррелей.')
print('38% скважин в регионе geo_2 содержат больше сырья, чем', round(geo[2]['product'].quantile(0.62),2), 'баррелей.')

36% скважин в регионе geo_0 содержат больше сырья, чем 111.87 баррелей.
17% скважин в регионе geo_1 содержат больше сырья, чем 110.99 баррелей.
38% скважин в регионе geo_2 содержат больше сырья, чем 111.34 баррелей.


Таким образом понятно, что если разрабатывать все скважины каждого региона с предложенным нам бюджетом, то это будет финансово невыгодно. Поэтому в каждом регионе и отбираются 200 наиболее перспективных скважин для рассчета максимальных прибылей от разработки в определенном регионе. 

## Расчёт прибыли и рисков 

In [21]:
model_geo = [model_geo_0, model_geo_1, model_geo_2]

In [22]:
## функция, рассчитывающая прибыль от 200 лучших скважин
def income(target_valid, predicted_valid):
    probs_sorted = pd.Series(predicted_valid).sort_values(ascending=False)
    selected = target_valid[probs_sorted.index][:TOP_200]
    return int((selected.sum() * PRICE_BARREL) - BUDGET)

Рассчитаем прибыль от наиболее перспективных скважин каждого региона.

In [23]:
for i in range(3):
    print('Прибыль от лучших 200 скважен в', i, 'регионе равна', 
          income(model_geo[i]['target_valid'], model_geo[i]['predicted_valid']))

Прибыль от лучших 200 скважен в 0 регионе равна 3320826043
Прибыль от лучших 200 скважен в 1 регионе равна 2415086696
Прибыль от лучших 200 скважен в 2 регионе равна 2710349963


Теперь определим 95% доверительный интервал для прибыли у каждого региона.

In [24]:
def confidence_interval(m):
    state = np.random.RandomState(125)
    values = []
    for i in range(1000):
        target_subsample = model_geo[m]['target_valid'].sample(n = 500, replace=True, random_state=state)
        probs_subsample = pd.Series(model_geo[m]['predicted_valid'])[target_subsample.index]
        values.append(income(target_subsample, probs_subsample))
    values = pd.Series(values)
    print('--- Регион', m, '---')
    print('Прибыль:', int(values.mean()))
    print('Вероятность убытков:', round(values.apply(lambda x: x < 0).sum() / len(values) * 100, 2), '%')
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    return int(lower), int(upper)

In [25]:
for i in range(3):
    print('95% доверительный интервал для', i, 'региона лежит между:', confidence_interval(i))
    print('\n*****************************************************************************************\n')

--- Регион 0 ---
Прибыль: 415512423
Вероятность убытков: 6.8 %
95% доверительный интервал для 0 региона лежит между: (-109898028, 947062942)

*****************************************************************************************

--- Регион 1 ---
Прибыль: 503903709
Вероятность убытков: 0.4 %
95% доверительный интервал для 1 региона лежит между: (105838585, 966480571)

*****************************************************************************************

--- Регион 2 ---
Прибыль: 417500453
Вероятность убытков: 7.5 %
95% доверительный интервал для 2 региона лежит между: (-168207000, 983313686)

*****************************************************************************************



### Вывод

Была поставлена задача выбрать регион добычи нефти с наименьший вероятностью убытков. Были предоставлены данные проб нефти(её качество и количество) из трех регионов(в каждом 10 000 месторождений).

После загрузки и подготовки данных, на них обучили модели для каждого региона и с необходимыми далее переменными сохранены.
В ходе обучения выяснилось, что у региона "1" очень низкое значение RMSE и R2-score равен единице.

При подготовке к рассчету прибыли выяснилось, что достаточный объём сырья для безубыточной разработки новой скважины равен 111.11 баррелей. При этом средний объём нефти в скважинах регионов "0" и "2" почти на треть превышает оный в регионе "1"(92.5 и 95 баррелей против 68.825). И скважин, которые содержат сырья для безубыточной разработки, от общего количества в каждом регионе равно: регион "0" - 36%, "1" - 17%, "2" - 38%.

Рассчет прибыли от 200 лучших скважин каждого региона показал следующее: 
- регион "0" - 3.320.826.043;
- регион "1" - 2.415.086.696;
- регион "2" - 2.710.349.963.

Далее, благодаря рассчету рисков были получены следующие данные:


| Регион | Прибыль | Вероятность убытков | 95% доверительный интервал |
|:--:|:--:|:--:|:--:|
| 0 | 415.512.423 | 6.8 % | (-109.898.028, 947.062.942) |  
| 1 | 503.903.709 | 0.4 % | (105.838.585, 966.480.571) |  
| 2 | 417.500.453 | 7.5 % | (-168.207.000, 983.313.686) |

**Только в регионе "1" вероятность убытков ниже 2,5%, а именно 0.4%. 
Соответственно к разработке рекомендуется регион "1" со средней предполагаемой прибылью в 503.903.709 и с доверительным интервалом (105.838.585, 966.480.571).**   
